In [1]:
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
import pandas as pd
from mp_api.client import MPRester

def calculate_uranium_density(formula, density):
    comp = Composition(formula)
    
    # Calculate the molar mass of the compound
    total_molar_mass = sum(Element(el).atomic_mass * amt for el, amt in comp.items())
    
    # Calculate the compound's volume
    volume = total_molar_mass / density if density else None
    
    # Calculate the uranium's molar mass
    uranium_mass = Element('U').atomic_mass * comp["U"]
    
    # Calculate the uranium density
    uranium_density = uranium_mass / volume if volume else None

    return uranium_density

with MPRester("API_key") as mpr:
    docs = mpr.summary.search(
        elements=["U"], is_stable=True, fields=["formula_pretty", "density"]
    )

    # Create a list of dictionaries, each representing a row in the dataframe
    data = []
    for doc in docs:
        row = {
            "Pretty Formula": doc.formula_pretty,
            "Density": doc.density,  # include density data
            "Uranium Density": calculate_uranium_density(doc.formula_pretty, doc.density)  # calculate uranium density
        }
        data.append(row)

    # Create dataframe
    df = pd.DataFrame(data)

df.to_csv("mp_uranium_compounds.csv", index=False)

Retrieving SummaryDoc documents:   0%|          | 0/783 [00:00<?, ?it/s]